# Creating FABnet IPv4 Ext Network: Manual Configuration

FABRIC provides a pair of layer 3 IP networking services with external access across every FABRIC site (FABnetv4Ext and FABnetv6Ext). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links and also provides external access. 

This notebook describes how to deploy the FABnetv4Ext service which is FABRIC's private IPv4 internet and configure the addresses manually, after the slice becomes active. The user can also enable/disable external access to specific IP addresses as requested.

Due to limited IPv4 address space, a single subnet is used across a site for all the slices requesting FABnetv4Ext.

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4Ext internet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add a separate `l3network` for each site and pass the list of interfaces on that site that you want to connect to FABnetv4Ext. All interfaces passed to `l3network` must be on the same site and each network will be placed on that site.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 

Manual configuration does not require any additional steps before the slice request is submitted. 


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 



In [ ]:
slice_name = 'MySlice'

[site1,site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]

# Node2
node2 = slice.add_node(name=node2_name, site=site2)
iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]

# NetworkS
net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4Ext')
net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4Ext')

#Submit Slice Request
slice.submit();

## Manually Configure IP Addresses

Most users will want to configure IP addresses on their new nodes.  FABlib provides some useful methods to help you manually configure basic IP addresses. 

### Get the Assigned Subnet

FABnetv4Ext networks are assigned a subnet and gateway by FABRIC.  You can get the subnet and available IPs from the FABlib objects. 

In [ ]:
slice=fablib.get_slice(name=slice_name)
network1 = slice.get_network(name=network1_name)
network1_available_ips = network1.get_available_ips()
network1.show()

network2 = slice.get_network(name=network2_name)
network2_available_ips =  network2.get_available_ips()
network2.show();

## Request external access
Users explicitly request the IP Adresses with external access. Due to limited IPv4 address space, a single subnet is used across a site for all the slices requesting FABnetv4Ext.


NOTE: The first IP on the subnet is requested to be made public. If it is already in use, the next available IP is assigned and returned to the user. If none of the external IPs are available, an error is returned.

In [ ]:
try:
    
    # Enable Public IPv4 make_ip_publicly_routable
    network1.make_ip_publicly_routable(ipv4=[str(network1_available_ips[0])])


    # Enable Public IPv4 make_ip_publicly_routable
    network2.make_ip_publicly_routable(ipv4=[str(network2_available_ips[0])])

    slice.submit()

except Exception as e:
    print(f"Exception: {e}")
    import traceback
    traceback.print_exc()

### Get latest Slice topology

In [ ]:
slice = fablib.get_slice(slice_name)

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available public IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Then set a route from *this network* to the *other network* or *external world* through the specified gateway. 

NOTE: This route should not be made default route ensuring the management traffic is not disrupted or misdirected.


Optionally, use the `node.execute()` method to show the results of adding the IP address and route.

NOTE: If by any chance, routes are mis-configured or you have lost management IP access to your VM, please invoke: `node.os_reboot()` to revert all configuration.


In [ ]:
node1 = slice.get_node(name=node1_name)        
node1_iface = node1.get_interface(network_name=network1_name)  
node1_addr = network1.get_public_ips()[0]
node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())

node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())

# Add route to external network
stdout, stderr = node1.execute()
stdout, stderr = node1.execute(f'sudo ip route add 0.0.0.0/0 via {network1.get_gateway()}')

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_device_name()}')    
stdout, stderr = node1.execute(f'ip route list')

### Configure Node2

Repeat the steps to add the next available IP to the second node and a route to the external network.

In [ ]:
node2 = slice.get_node(name=node2_name)        
node2_iface = node2.get_interface(network_name=network2_name) 
node2_addr = network2.get_public_ips()[0]
node2_iface.ip_addr_add(addr=node2_addr, subnet=network2.get_subnet())

node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())

# Add route to external network
stdout, stderr = node2.execute(f'sudo ip route add 0.0.0.0/0 via {network2.get_gateway()}')


stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_device_name()}')
stdout, stderr = node2.execute(f'ip route list')

## Run the Experiment

We will find the ping round trip time for this pair of sites and also ping round trip to an external website: `bing.com`.  Your experiment should be more interesting!


In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()
node1_addr = node1.get_interface(network_name=network1_name).get_ip_addr()

In [ ]:
import subprocess

def ping_ip(ip_address):
    try:
        result = subprocess.run(['ping', '-c', '4', ip_address], capture_output=True, text=True, check=True)
        print(result.stdout)
        return True
    except subprocess.CalledProcessError:
        return False

### Ping Node1

In [ ]:
ping_ip(str(node1_addr))

### Ping Node2

In [ ]:
ping_ip(str(node2_addr))

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.delete()